In [1]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('./data/차량_연비_데이터(20022022).csv')

# 데이터 확인
df.head()

,YEAR,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,CITY (L/100 km),HWY (L/100 km),COMB (L/100 km),COMB (mpg),EMISSIONS
0,2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,9.2,6.7,8.1,35,186
1,2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,8.5,6.5,7.6,37,175
2,2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,12.2,7.4,10.0,28,230
3,2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,13.4,9.2,11.5,25,264
4,2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,10.0,7.0,8.6,33,198


In [ ]:
# 데이터 정보 확인
df.info()

In [ ]:
# 결측치 확인
df.isna().sum()

In [ ]:
# 수치형 컬럼 기술통계 확인
df.describe()

# 전처리 전략
1. km단위 조정(파생 변수 생성) : 한국인이 이해하기 쉽게 조정
2. mpg열 및 새로 추가된 컬럼 제거 : mpg는 mile per gallan으로 미국 단위라 (L/100 km를 조정해서 사용)
3. 차량 분류 대소문자 통일, 특수문자 정제
4. 차량 분류 매핑 정리 : SPECIAL PURPOSE VEHICLE 모델명 구글링해보니, van과 비슷하게 생겼음 그냥 밴임, SUV가 SUV-STANDARD와 비슷한 
    특징이라 SUV-STANDARD로 정리
5. 컬럼 별 변수 확인 및 IQR 값으로 이상치 처리안함 # 극단값이 분석에 필요하기 때문(16실린더같은 값이 연비가 안좋다라는 것이 필요함)
6. fuel값 이름 조정 : 가독성 상향 # 발표 시에만 표시
7. 



In [2]:
# 1. 단위 변환하여 새 컬럼 생성: L/100km → km/L
df['COMB (km/L)'] = 100 / df['COMB (L/100 km)']
df['CITY (km/L)'] = 100 / df['CITY (L/100 km)']
df['HWY (km/L)']  = 100 / df['HWY (L/100 km)']
df.head()

,YEAR,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,CITY (L/100 km),HWY (L/100 km),COMB (L/100 km),COMB (mpg),EMISSIONS,COMB (km/L),CITY (km/L),HWY (km/L)
0,2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,9.2,6.7,8.1,35,186,12.345679,10.869565,14.925373
1,2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,8.5,6.5,7.6,37,175,13.157895,11.764706,15.384615
2,2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,12.2,7.4,10.0,28,230,10.000000,8.196721,13.513514
3,2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,13.4,9.2,11.5,25,264,8.695652,7.462687,10.869565
4,2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,10.0,7.0,8.6,33,198,11.627907,10.000000,14.285714


In [3]:
# 2. COMB(mpg) 등 필요없는 컬럼들 삭제하기
df = df.drop(columns=[
    'COMB (L/100 km)', 'CITY (L/100 km)', 'HWY (L/100 km)', 'COMB (mpg)'
])
df.head()

,YEAR,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,EMISSIONS,COMB (km/L),CITY (km/L),HWY (km/L)
0,2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,186,12.345679,10.869565,14.925373
1,2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,175,13.157895,11.764706,15.384615
2,2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,230,10.000000,8.196721,13.513514
3,2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,264,8.695652,7.462687,10.869565
4,2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,198,11.627907,10.000000,14.285714


In [4]:
# 3. 클래스명 대문자로 통일하기
df["VEHICLE CLASS"] = df["VEHICLE CLASS"].str.strip().str.upper()
df['VEHICLE CLASS'] = df['VEHICLE CLASS'].str.replace(':', ' -', regex=False)
df["VEHICLE CLASS"].unique()

<StringArray>
[                 'COMPACT',                 'MID-SIZE',
               'SUBCOMPACT', 'STATION WAGON - MID-SIZE',
              'MINICOMPACT',               'TWO-SEATER',
    'STATION WAGON - SMALL',                'FULL-SIZE',
                      'SUV',              'VAN - CARGO',
          'VAN - PASSENGER',  'PICKUP TRUCK - STANDARD',
     'PICKUP TRUCK - SMALL',                  'MINIVAN',
           'SUV - STANDARD',  'SPECIAL PURPOSE VEHICLE',
              'SUV - SMALL']
Length: 17, dtype: str

In [5]:
# 4. 차량 분류 그룹핑
mapping = {
     # SUV
    'SUV': 'SUV - STANDARD',
    'SPECIAL PURPOSE VEHICLE': 'VAN - CARGO'
}
group_df = df.copy()
group_df['VEHICLE CLASS'] = df['VEHICLE CLASS'].replace(mapping).copy()
group_df["VEHICLE CLASS"].unique()



<StringArray>
[                 'COMPACT',                 'MID-SIZE',
               'SUBCOMPACT', 'STATION WAGON - MID-SIZE',
              'MINICOMPACT',               'TWO-SEATER',
    'STATION WAGON - SMALL',                'FULL-SIZE',
           'SUV - STANDARD',              'VAN - CARGO',
          'VAN - PASSENGER',  'PICKUP TRUCK - STANDARD',
     'PICKUP TRUCK - SMALL',                  'MINIVAN',
              'SUV - SMALL']
Length: 15, dtype: str

In [6]:
group_df.head()

,YEAR,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,EMISSIONS,COMB (km/L),CITY (km/L),HWY (km/L)
0,2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,186,12.345679,10.869565,14.925373
1,2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,175,13.157895,11.764706,15.384615
2,2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,230,10.000000,8.196721,13.513514
3,2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,264,8.695652,7.462687,10.869565
4,2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,198,11.627907,10.000000,14.285714


In [10]:
# 전처리한 데이터 저장
group_df.to_csv("data/processed_df.csv", index=False)
